In [1]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("RFP.pdf")
docs = loader.load_and_split()
docs[0]

Document(metadata={'source': 'RFP.pdf', 'page': 0}, page_content='DETROIT WAYNE INTEGRATED HEALTH NETWORK  \n \nREQUEST FOR PROPOSALS \nFOR  \nDEVELOPMENT AND IMPLEMENTATION OF AI MODELS TO REDUCE HOSPITAL RECIDIVISM  \n \nCONTROL NO. 20 25-002 \n  \n \nIssue Date: August 13, 2024 \n \nPre-Proposal Virtual Conference:  Wednesday , September 4, 2024 at 10:00 a.m. EDT  \nLink to Reg ister: DWIHN Pre -Proposal Conference   \n  \n     \nPre-Proposal Question Deadline:   Thursday, September  5, 2024 at 4:00 p.m. EDT  \n Proposal Deadline:\n Friday, September 27, 2024 at 4:00 p.m. EDT   \n \nProposal Submission Format:  Electronic  \n \nPurchasing Contact: Jean Mira, Procurement Administrator  \n Email:  jmira1@dwihn.org  \n Purchasing Division  \n Email: procurement@dwihn.org   \n Description: The Detroit Wayne Integrated Health Network  (DWIHN) is requesting proposals for development \nand implementation of artificial intelligence ( AI) models to reduce hospital recidivism . \n A copy of t

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

final_documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
final_documents

[Document(metadata={'source': 'RFP.pdf', 'page': 0}, page_content='DETROIT WAYNE INTEGRATED HEALTH NETWORK  \n \nREQUEST FOR PROPOSALS \nFOR  \nDEVELOPMENT AND IMPLEMENTATION OF AI MODELS TO REDUCE HOSPITAL RECIDIVISM  \n \nCONTROL NO. 20 25-002 \n  \n \nIssue Date: August 13, 2024 \n \nPre-Proposal Virtual Conference:  Wednesday , September 4, 2024 at 10:00 a.m. EDT  \nLink to Reg ister: DWIHN Pre -Proposal Conference   \n  \n     \nPre-Proposal Question Deadline:   Thursday, September  5, 2024 at 4:00 p.m. EDT  \n Proposal Deadline:\n Friday, September 27, 2024 at 4:00 p.m. EDT   \n \nProposal Submission Format:  Electronic  \n \nPurchasing Contact: Jean Mira, Procurement Administrator  \n Email:  jmira1@dwihn.org  \n Purchasing Division  \n Email: procurement@dwihn.org   \n Description: The Detroit Wayne Integrated Health Network  (DWIHN) is requesting proposals for development \nand implementation of artificial intelligence ( AI) models to reduce hospital recidivism .'),
 Document(

In [6]:
len(final_documents)

62

In [3]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv

load_dotenv()
GROQ_API_KEY=os.getenv("GROQ_API_KEY")

llm=ChatGroq(api_key=GROQ_API_KEY,model="Llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000232E50558D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000232E398CF90>, model_name='Llama3-8b-8192', groq_api_key=SecretStr('**********'))

In [7]:
from langchain import PromptTemplate
chunks_prompt="""
Please summarize the below speech:
Speech:`{text}'
Summary:
"""
map_prompt_template=PromptTemplate(input_variables=['text'],
                                    template=chunks_prompt)

In [8]:
final_prompt='''
Provide the final summary of the entire speech with these important points.
Add a Motivation Title,Start the precise summary with an introduction and provide the summary in number 
points for the speech.
Speech:{text}

'''
final_prompt_template=PromptTemplate(input_variables=['text'],template=final_prompt)
final_prompt_template

PromptTemplate(input_variables=['text'], template='\nProvide the final summary of the entire speech with these important points.\nAdd a Motivation Title,Start the precise summary with an introduction and provide the summary in number \npoints for the speech.\nSpeech:{text}\n\n')

In [10]:
from langchain.chains.summarize import load_summarize_chain
summary_chain=load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    map_prompt=map_prompt_template,
    combine_prompt=final_prompt_template,
    verbose=True
)

output=summary_chain.run(final_documents)
output



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Please summarize the below speech:
Speech:`DETROIT WAYNE INTEGRATED HEALTH NETWORK  
 
REQUEST FOR PROPOSALS 
FOR  
DEVELOPMENT AND IMPLEMENTATION OF AI MODELS TO REDUCE HOSPITAL RECIDIVISM  
 
CONTROL NO. 20 25-002 
  
 
Issue Date: August 13, 2024 
 
Pre-Proposal Virtual Conference:  Wednesday , September 4, 2024 at 10:00 a.m. EDT  
Link to Reg ister: DWIHN Pre -Proposal Conference   
  
     
Pre-Proposal Question Deadline:   Thursday, September  5, 2024 at 4:00 p.m. EDT  
 Proposal Deadline:
 Friday, September 27, 2024 at 4:00 p.m. EDT   
 
Proposal Submission Format:  Electronic  
 
Purchasing Contact: Jean Mira, Procurement Administrator  
 Email:  jmira1@dwihn.org  
 Purchasing Division  
 Email: procurement@dwihn.org   
 Description: The Detroit Wayne Integrated Health Network  (DWIHN) is requesting proposals for development 
and implementation of artificial intelligen

c:\Users\Rengarajan\POC\TextSummarization\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
c:\Users\Rengarajan\POC\TextSummarization\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (8953 > 1024). Running this sequence through the model wil



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Provide the final summary of the entire speech with these important points.
Add a Motivation Title,Start the precise summary with an introduction and provide the summary in number 
points for the speech.
Speech:Here is a summary of the speech:

The Detroit Wayne Integrated Health Network (DWIHN) is requesting proposals from vendors to develop and implement artificial intelligence (AI) models to reduce hospital recidivism. The request for proposals includes a pre-proposal virtual conference on September 4, 2024, and a deadline for submitting proposals on September 27, 2024.

Here is a summary of the speech:

The Detroit Wayne Integrated Health Network (DWIHN) is seeking proposals from developers to create and implement artificial intelligence (AI) models to reduce hospital readmissions. The request for proposal (RFP) can be obtained from the BidNet Direct website until the deadline

'Here is the final summary of the entire speech with the important points:\n\n**Motivation Title:** "Embracing Artificial Intelligence to Reduce Hospital Readmissions"\n\n**Summary in Number Points:**\n\n1. Introduction: The Detroit Wayne Integrated Health Network (DWIHN) is seeking proposals from developers to create and implement artificial intelligence (AI) models to reduce hospital readmissions.\n\n2. Request for Proposal (RFP): The RFP can be obtained from the BidNet Direct website until the deadline. Bidders are responsible for checking the website for updates and addenda until the deadline. Late proposals will not be accepted.\n\n3. Proposal Guidelines: Proposers should submit their questions in writing to the Purchasing Contact before the Pre-Proposal Question Deadline. All answers to these questions will be issued in the form of written addenda.\n\n4. Virtual Pre-Proposal Meeting: A virtual Pre-Proposal meeting will take place on September 4, 2024, at 10:00 a.m. EDT, and Propo